# CHEST X-RAY Pneumonia Classification with EfficientNet

This notebook ports the experiment shared by **nikoneri**. It leverages TensorFlow's EfficientNetB0 backbone with an unfreezing stage for fine-tuning on the chest X-ray dataset.

In [ ]:
from pathlib import Path

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.preprocessing import image_dataset_from_directory

DATA_ROOT = Path('..') / 'data' / 'chest_xray'
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
AUTOTUNE = tf.data.AUTOTUNE

## Build `tf.data` datasets

Using `image_dataset_from_directory` keeps the pipeline close to the original implementation while providing efficient, batched datasets compatible with mixed precision.

In [ ]:
train_ds = image_dataset_from_directory(
    DATA_ROOT / 'train',
    label_mode='binary',
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=True,
    seed=123
)

val_ds = image_dataset_from_directory(
    DATA_ROOT / 'val',
    label_mode='binary',
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=False
)

test_ds = image_dataset_from_directory(
    DATA_ROOT / 'test',
    label_mode='binary',
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=False
)

class_names = train_ds.class_names
print(class_names)

## Prefetch and augment

Basic augmentation replicates the behavior of the published notebook and improves generalization.

In [ ]:
normalization_layer = layers.Rescaling(1.0 / 255)

data_augmentation = models.Sequential([
    layers.RandomFlip('horizontal'),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1)
])

train_ds = train_ds.map(lambda x, y: (data_augmentation(normalization_layer(x)), y), num_parallel_calls=AUTOTUNE)
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y), num_parallel_calls=AUTOTUNE)
test_ds = test_ds.map(lambda x, y: (normalization_layer(x), y), num_parallel_calls=AUTOTUNE)

train_ds = train_ds.prefetch(AUTOTUNE)
val_ds = val_ds.prefetch(AUTOTUNE)
test_ds = test_ds.prefetch(AUTOTUNE)

## Create the EfficientNet model

We start with a frozen backbone for rapid convergence, then unfreeze the top layers for fine-tuning.

In [ ]:
base_model = EfficientNetB0(include_top=False, weights='imagenet', input_shape=IMG_SIZE + (3,))
base_model.trainable = False

inputs = layers.Input(shape=IMG_SIZE + (3,))
x = base_model(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(1, activation='sigmoid')(x)

model = models.Model(inputs, outputs)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

## Train the frozen backbone

Training the classifier head establishes a strong initialization before unfreezing.

In [ ]:
initial_history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=5
)

## Fine-tune the network

We unfreeze the last few blocks and continue training with a lower learning rate.

In [ ]:
base_model.trainable = True
for layer in base_model.layers[:-40]:
    layer.trainable = False

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
              loss='binary_crossentropy',
              metrics=['accuracy'])

fine_tune_history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=15
)

## Evaluate on the test dataset

The final accuracy helps compare across different contributor notebooks.

In [ ]:
results = model.evaluate(test_ds)
print(dict(zip(model.metrics_names, results)))

## Save the model weights

Persisting the trained weights makes it easy to deploy the classifier elsewhere.

In [ ]:
model.save('nikoneri_efficientnet_pneumonia')